In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import os
from nltk.corpus import stopwords
import math
import random
from sklearn.feature_extraction.text import TfidfVectorizer


# First Round: estimating label candidates

In [2]:
# Set the file path
file_path = "/home/marcnaweb/code/marcnaweb/car_recommendation_engine/raw_data/text_files/extracted_datasheets/"


In [3]:
files = os.listdir(file_path)
random.shuffle(files) # shuffle the files to get a random sample
text_files = files[1000:1030]  # [ f"{file_path}1.txt", f"{file_path}2.txt", f"{file_path}3.txt",  f"{file_path}4.txt" ,  f"{file_path}5.txt"]

In [4]:
text_content = []

for file in text_files:
    with open(f"{file_path}{file}", 'r') as file:
        content = file.read()
        try:
            content = content.split('Ficha Técnica')[1]
            content = content.split('Fotos')[0]
            #text_content.append(content)
            #text_content.append(content)
            content = content.split('\n')
            #print(content)


        except:
            pass
    temp = []
    for line in content:
        #pass
        # Remove words in between brackets
        line = re.sub(r'\[.*?\]', ' ', line)
        line = re.sub(r'\CNW', ' ', line)
        words = line.split()
        # Remove words that contain a special character
        words = [word for word in words if all(  c == '\n' or c == '/'  or c =='.' or c == ',' for c in string.punctuation if c in word)]

        #include a position indicator to the words that are not stopwords
        # from left to right
        words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
        # from right to left
        words.reverse()
        words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
        #reput the words in the right order
        words.reverse()

        cleaned_line = ' '.join(words)
        #print(cleaned_line)
        text_content.append(cleaned_line)

#print(text_content)


In [5]:
words = ["de", '12,3.5', 'a', 'bb']


words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]

#words.reverse()
#words = [f"{i}_{word}" for i, word in enumerate(words)]
#words.reverse()

words

['de', '1_12,3.5', 'a', '3_bb']

In [6]:
enumerate(words)

In [7]:

corpus = text_content
vectorizer = TfidfVectorizer(min_df=1 , stop_words=stopwords.words('portuguese') , token_pattern=r'\b[\w_,.]*[_,.][\w_,.]*\b' )  #
X = vectorizer.fit(corpus)
vectorizer.transform(corpus).toarray()
vectorizer.get_feature_names_out()

weighted_words = pd.DataFrame(vectorizer.fit_transform(corpus).toarray(),
                 columns = vectorizer.get_feature_names_out(), index = corpus)

weighted_words.reset_index(inplace=True)
weighted_words["temp"] = weighted_words["index"].apply(lambda x: len(x.split()))
weighted_words =    weighted_words[weighted_words["temp"] > 0 ].drop(columns=["temp"])
weighted_words.reset_index(inplace=True)
weighted_words.drop(columns=["level_0"], inplace=True)
#weighted_words


In [8]:
#weighted_words[weighted_words.index == 5245]['index'].values[0]

In [9]:

def get_top_words(line_weight, cutting_value):
    print(cutting_value)
    top_words = line_weight[line_weight[line_number] > cutting_value]
    #print(top_words.index.tolist())
    value_to_return = top_words.index.tolist()
    value_to_return.reverse()
    return value_to_return  #we reverse the list to get the less important words first


In [10]:
a = ["a","b", "c"]#.reverse()
a.reverse()
print(a)

['c', 'b', 'a']


In [11]:



def get_label_and_value(line, value_words):
    # Get the label and the value
    #print(line)

    # Create a list of words

    words = value_words
    #print(words)

    # Create a regex pattern to match the string between any of the words, including the words
    #pattern = r'({}).*?'.format('|'.join(words)) * len(words)
    #pattern = r'({}\S*)?'.format('|'.join(words)) * len(words)
    #pattern = r'({}\S*)'.format('|'.join(words))
    #print(pattern)
    #pattern = r'({}[^\s]*)'.format('|'.join(words))  * len(words)
    pattern = r'({}[^\s]*).*?'.format('|'.join(words)) * len(words)


    # Find the string between the words
    match = re.search(pattern, line, re.DOTALL)
    #match = re.findall(pattern, line, re.DOTALL)

    print(match)


    if match:
        # get the value words
        result = match.group()
        #clean the string from position indicators _0, _1, _2, etc
        pattern = r'\b\w*_'
        result = re.sub(pattern, '', result)
        print(f"found value: {result}")

        # remove the value words from the line to get the label
        #label = line
        #for word in match.group().split():
        #    if word not in stopwords.words('portuguese'):
        #        label = label.replace(word, '')
        #        label = label.replace('  ', ' ') # remove double spaces

        #remove the result from line to get the label
        line = re.sub(pattern, '', line)
        label = line.replace(result, '')
        label = label.replace('  ', ' ') # remove double spaces

        #clean the string from position indicators _0, _1, _2, etc
        pattern = r'\b\w*_'
        label = re.sub(pattern, '', label)
        print("found label: " + label)



        return label, result

    else:
        print('No match found')






#print(line)
#get_label_and_value(line.lower(), top_words)

In [12]:
#get_label_and_value("Torque máximo 22,1 kgfm".lower(), [  'máximo', '22' ])
"okok sffsfsf".replace("sf", "")

'okok f'

In [13]:
#test line numbers: 5515, 270, 5245, 562

label_values_list = []

for line_number in weighted_words.index.tolist():
    print(f"line_number: {line_number}")

    line = weighted_words[weighted_words.index == line_number]['index'].values[0]

    if len(line.split()) > 1:
        print(line)

        #get line weight
        line_weight = weighted_words[weighted_words.index == line_number].select_dtypes(include=[np.number]).T.sort_values(line_number, ascending=False)

        #get cutting value =  mean + (standard deviation) divided by (count)
        cutting_value = line_weight[line_weight[line_number] > 0].describe().loc['mean'] + line_weight[line_weight[line_number] > 0].describe().loc['std'] / line_weight[line_weight[line_number] > 0].describe().loc['count']
        cutting_value = cutting_value.values[0]

        top_words = get_top_words(line_weight, cutting_value)
        label_values_list.append(get_label_and_value(line.lower(), top_words))



#print the cutting value
#print(f"Cutting_value: {cutting_value}")
#print(line_weight.head(10))
#print(line_weight[line_weight[line_number] > 0].describe())


line_number: 0
3_0_Fiat 2_1_Ducato 1_2_Chassi 0_3_2.3
0.5
<re.Match object; span=(0, 0), match=''>
found value: 
found label: fiat ducato chassi 2.3
line_number: 1
1_0_Ano 0_1_2019
0.7688482144900338
<re.Match object; span=(8, 16), match='0_1_2019'>
found value: 2019
found label: ano 
line_number: 2
1_0_Preço 0_1_160.158
0.7688482144900338
<re.Match object; span=(10, 21), match='0_1_160.158'>
found value: 160.158
found label: preço 
line_number: 3
line_number: 4
1_0_Propulsão 0_1_Combustão
0.708326392480367
<re.Match object; span=(14, 27), match='0_1_combustão'>
found value: combustão
found label: propulsão 
line_number: 5
1_0_Combustível 0_1_Diesel
0.7611699434364244
<re.Match object; span=(16, 26), match='0_1_diesel'>
found value: diesel
found label: combustível 
line_number: 6
1_0_Revisões 0_1_5.340
0.7403720595950377
<re.Match object; span=(13, 22), match='0_1_5.340'>
found value: 5.340
found label: revisões 
line_number: 7
line_number: 8
line_number: 9
1_0_Procedência 0_1_Importad

# Second round: refining found labels

In [14]:
label_values_list_temp = label_values_list.copy()
label_values_list_temp = [x for x in label_values_list_temp if x is not None]
i = 0
for label, value in label_values_list_temp:
    #removing stop words from label
    label_words = label.split()
    label_words = [f"{word}" if word not in stopwords.words('portuguese') else '' for i, word in enumerate(label_words)]
    label = ' '.join(label_words)
    #remove double spaces
    label = label.replace('  ', ' ')
    label_values_list_temp[i] = (label, value)
    i+=1
label_values_list_temp

[('fiat ducato chassi 2.3', ''),
 ('ano', '2019'),
 ('preço', '160.158'),
 ('propulsão', 'combustão'),
 ('combustível', 'diesel'),
 ('revisões', '5.340'),
 ('procedência', 'importado'),
 ('garantia', '1 ano'),
 ('porte', 'grande'),
 ('lugares', '2'),
 ('portas', '2'),
 ('geração', '3'),
 ('índice cnw', '51,39'),
 ('ranking cnw', '16468'),
 ('nota leitor', '7,0'),
 ('instalação', 'dianteiro'),
 ('disposição', 'transversal'),
 ('aspiração', 'turbocompressor'),
 ('alimentação injeção', 'direta'),
 ('cilindros linha', '4'),
 ('comando válvulas cabeçote', 'duplo'),
 ('acionam. comando correia dentada', ''),
 ('tuchos', 'hidráulicos'),
 ('válvulas cilindro', '4'),
 ('diâmetro cilindro mm', '88'),
 ('curso pistão mm', '94'),
 ('razão compressão', ''),
 ('código motor', 'multijet f1a'),
 ('cilindrada unitária', '572'),
 ('deslocamento', '2287'),
 ('potência máxima cv', '130'),
 ('regime potência máx. rpm', '3600'),
 ('torque máximo kgfm', '32,6'),
 ('regime torque máx. rpm', '1800'),
 ('peso/p

In [29]:
label_values_df = pd.DataFrame(label_values_list_temp, columns=["label", "value"])

"""
    label_values_df.sort_values(by='label_frequency', ascending=False, inplace=True)
    label_values_df.drop_duplicates(subset='label', keep='first', inplace=True)
    pd.set_option('display.max_rows', 200)


    #label_values_df['label'].values.tolist()
    label_values_df#.iloc[500:621]
    #label_values_df.describe()
    #text_content = label_values_df['label'].values.tolist()
    #text_content
    label_values_df[label_values_df['label_frequency'] > 2]#.describe()

    #label_values_df[label_values_df['label'] == "equipamento série"].drop_duplicates(subset='value', keep='first', inplace=False)#.describe()

"""
label_values_df.head(200)

,label,value
0,fiat ducato chassi 2.3,
1,ano,2019
2,preço,160.158
3,propulsão,combustão
4,combustível,diesel
5,revisões,5.340
6,procedência,importado
7,garantia,1 ano
8,porte,grande
9,lugares,2


# Filtering found labels to keep only common words

In [16]:
# we choose to keep labels that does not contain numbers

def remove_numbers(text):
    words =  text.split()
    filtered_words = [word for word in words if re.match(r'\D*$', word)]
    filtered_text = ' '.join(filtered_words)
    #print(filtered_text)
    return(filtered_text)


In [28]:
label_values_df['filtered_label'] = label_values_df['label'].apply(lambda x: remove_numbers(x))
label_values_df = label_values_df[label_values_df["filtered_label"] != "" ]  #.dropna(subset=['filtered_label'])
label_values_df.head(200)

/tmp/ipykernel_41925/2036597908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_values_df['filtered_label'] = label_values_df['label'].apply(lambda x: remove_numbers(x))


,label,value,filtered_label,filtered_label_frequency
1767,equipamento série,sensores de estacionamento dianteiro,equipamento série,929
1012,altura flanco mm,172,altura flanco mm,50
1329,elemento elástico mola,helicoidal,elemento elástico mola,48
1395,equipamento,série,equipamento,38
1625,equipamento opcional,chave presencial,equipamento opcional,27
1537,litros,495,litros,25
2209,procedência,importado,procedência,25
1912,aceleração km/h s,"20,6",aceleração km/h s,25
383,distância mm,2939,distância mm,25
2850,traseiros r16,215/65,traseiros,25


In [18]:
label_values_df["filtered_label_frequency"] = label_values_df.groupby('filtered_label')['filtered_label'].transform('count')
label_values_df

/tmp/ipykernel_41925/145380081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_values_df["filtered_label_frequency"] = label_values_df.groupby('filtered_label')['filtered_label'].transform('count')


,label,value,filtered_label,filtered_label_frequency
0,fiat ducato chassi 2.3,,fiat ducato chassi,1
1,ano,2019,ano,25
2,preço,160.158,preço,25
3,propulsão,combustão,propulsão,25
4,combustível,diesel,combustível,25
...,...,...,...,...
2990,aceleração km/h s,"14,7",aceleração km/h s,25
2991,urbano km/l,"5,3",urbano km/l,24
2992,rodoviário km/l,"8,3",rodoviário km/l,24
2993,urbana km,445,urbana km,24


In [27]:
label_values_df.sort_values(by='filtered_label_frequency', ascending=False , inplace=True)
label_values_df.drop_duplicates(subset='filtered_label', keep='first', inplace=True)
pd.set_option('display.max_rows', 200)


#label_values_df['label'].values.tolist()
label_values_df#.iloc[500:621]
#label_values_df.describe()
#text_content = label_values_df['label'].values.tolist()
#text_content
label_values_df[label_values_df['filtered_label_frequency'] > 2][["filtered_label"]]#.describe()

#label_values_df[label_values_df['label'] == "equipamento série"].drop_duplicates(subset='value', keep='first', inplace=False)#.describe()
#label_values_df[["filtered_label"]]

/tmp/ipykernel_41925/1079329229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_values_df.sort_values(by='filtered_label_frequency', ascending=False , inplace=True)
/tmp/ipykernel_41925/1079329229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_values_df.drop_duplicates(subset='filtered_label', keep='first', inplace=True)


,filtered_label
1767,equipamento série
1012,altura flanco mm
1329,elemento elástico mola
1395,equipamento
1625,equipamento opcional
1537,litros
2209,procedência
1912,aceleração km/h s
383,distância mm
2850,traseiros


In [3]:
label_values_df

NameError: name 'label_values_df' is not defined

# STOP HERE 

#

In [ ]:
import re

# assuming data is your list of strings
data = ['traseira', 'ind  1ep.', 'km/l', 'str5ing4', '5', '']

# filter out strings that contain numbers
filtered_data = [s for s in data if re.match(r'\D*$', s)]

print(filtered_data)  # Output: ['string1', 'string2']

In [ ]:
''.split()

In [ ]:
import re

# assuming data is your list of strings
data = ['traseira', 'indep.', 'km/l', 'string4', '5']

# filter out strings that contain numbers
filtered_data = [s for s in data if re.match(r'\D*$', s)]

print(filtered_data)  # Output: ['string1', 'string2']

In [ ]:
label_list = label_values_df['label'].values.tolist()  #skipping as the labels  is already cleaned
label_list

In [ ]:

corpus = label_list
# similar sript as above

vectorizer = TfidfVectorizer()  #min_df=1 , stop_words=stopwords.words('portuguese') , token_pattern=r'\b[\w_,.]*[_,.][\w_,.]*\b' )  #
X = vectorizer.fit(corpus)
vectorizer.transform(corpus).toarray()
vectorizer.get_feature_names_out()

weighted_words = pd.DataFrame(vectorizer.fit_transform(corpus).toarray(),
                 columns = vectorizer.get_feature_names_out(), index = corpus)

weighted_words.reset_index(inplace=True)
weighted_words["temp"] = weighted_words["index"].apply(lambda x: len(x.split()))
weighted_words =    weighted_words[weighted_words["temp"] > 0 ].drop(columns=["temp"])
weighted_words.reset_index(inplace=True)
weighted_words.drop(columns=["level_0"], inplace=True)
#weighted_words


In [ ]:
weighted_words

In [ ]:
#similar as above: changed the cutting value

label_values_list = []

for line_number in weighted_words.index.tolist():
    print(f"line_number: {line_number}")

    line = weighted_words[weighted_words.index == line_number]['index'].values[0]

    if len(line.split()) > 0:
        print(line)

        #get line weight
        line_weight = weighted_words[weighted_words.index == line_number].select_dtypes(include=[np.number]).T.sort_values(line_number, ascending=False)

        #get cutting value =  mean - (standard deviation) divided by (count)
        cutting_value = line_weight[line_weight[line_number] > 0].describe().loc['mean'] - line_weight[line_weight[line_number] > 0].describe().loc['std'] / line_weight[line_weight[line_number] > 0].describe().loc['count']
        cutting_value = cutting_value.values[0]

        top_words = get_top_words(line_weight, cutting_value)
        label_values_list.append(get_label_and_value(line.lower(), top_words))


refined_label_values_list = label_values_list.copy()

#print the cutting value
#print(f"Cutting_value: {cutting_value}")
#print(line_weight.head(10))
#print(line_weight[line_weight[line_number] > 0].describe())


In [ ]:
refined_label_values_list

In [ ]:
refined_label_values_df = pd.DataFrame(refined_label_values_list, columns=["label", "value"])
refined_label_values_df
refined_label_values_df['label_frequency'] =   refined_label_values_df['label'].map(refined_label_values_df['label'].value_counts())
refined_label_values_df['label_frequency'] =    refined_label_values_df['label_frequency']   +  refined_label_values_df['label'].map(refined_label_values_df['value'].value_counts())  #refined_label_values_df['label_frequency'] + 1/(refined_label_values_df['label_frequency'] + 1)
refined_label_values_df.sort_values(by='label_frequency', ascending=False, inplace=True)
refined_label_values_df.drop_duplicates(subset=['label', 'value'], keep='first', inplace=True)
refined_label_values_df


In [ ]:
#re-starting the process
content = label_values_df['label'].values.tolist()
text_content = []

temp = []
for line in content:
    #pass
    # Remove words in between brackets
    line = re.sub(r'\[.*?\]', ' ', line)
    words = line.split()
    # Remove words that contain a special character
    words = [word for word in words if all(  c == '\n' or c == '/'  or c =='.' or c == ',' for c in string.punctuation if c in word)]

    #include a position indicator to the words that are not stopwords
    # from left to right
    words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
    # from right to left
    words.reverse()
    words = [f"{i}_{word}" if word not in stopwords.words('portuguese') else word for i, word in enumerate(words)]
    #reput the words in the right order
    words.reverse()

    cleaned_line = ' '.join(words)
    #print(cleaned_line)
    text_content.append(cleaned_line)


In [ ]:
weighted_words[weighted_words.index == 100]['index'].values[0]

In [ ]:
line_number = 41
line = weighted_words[weighted_words.index == line_number ]['index'].values[0]
line_weight = weighted_words[weighted_words.index == line_number].select_dtypes(include=[np.number]).T.sort_values(line_number, ascending=False)

#get cutting value =  mean + (standard deviation) divided by (count)
cutting_value = line_weight[line_weight[line_number] > 0].describe().loc['mean'] - line_weight[line_weight[line_number] > 0].describe().loc['std'] / line_weight[line_weight[line_number] > 0].describe().loc['count']
cutting_value = cutting_value.values[0]

top_words = get_top_words(line_weight, cutting_value)
print(top_words)
print(line )
get_label_and_value(line.lower(), top_words)

print(line_weight[line_weight[line_number] > 0].describe())
line_weight.head(10)


In [ ]:
import re
pattern = r"(?u)\b\w\w+\b|\b\d{1,2}[.,]?\d*\b|\b\d\b" #old

pattern = r'\b[a-zA-Z0-9.,]{1,}\b|\b\d\b'

pattern = r"(?u)\b\w\w+\b|\b\d{1,2}[.,]?\d*\b|\b\d\b"

pattern = r'\b[a-zA-Z]{2,}\b|\b\d\b|\b\d{1,2}[.,]?\d*\b'

pattern = r"(?<=\s|^)[a-zA-Z]{2,}(?=\s|$)|(?<=\s|^)\d(?=\s|$)|(?<=\s|^)\d{1,2}[.,]?\d*(?=\s|$)"

pattern = r'\b[a-zA-Z]{2,}\b|\b\d\b|\b\d{1,2}[.,]\d*\b'

pattern = r'\b\w*_\w*\b'

pattern = r'\b[\w_,.]*[_,.][\w_,.]*\b'

pattern = r'\b[\w_,.]*[_,.][\w_,.]*\b'   #used by tdidf vectorizer

text = "0_Torque 1_máximo 2_16,5 3_kgfm 4_a 5_4.500 6_rpm 7_em 8_4 9_cilindros"

text = "2_0_Área 1_1_frontal 0_2_3 "
matches = re.findall(pattern, text)

print(matches)  # Output: ['12', 'with', 'and', '3']


In [ ]:
import re

text = "This is a_test_string but not this"
pattern = r'\b\w*_'
substituted_text = re.sub(pattern, '', text)

print(substituted_text)  # Output: 'This is string but not this'

In [ ]:
import re

text = "0_Torque 1_máximo 2_16,5 3_kgfm 4_a 5_4.500 6_rpm 7_em 8_4 9_cilindros"
pattern = r'\b\w*_'
substituted_text = re.sub(pattern, '', text)

print(substituted_text)  # Output: 'This is string but not this'

In [ ]:
import re

# Create a sample text
text = 'equipamento de série capô sustentado por mola a...'.lower()

# Create a list of words
words = top_words

# Create a regex pattern to match the string between any of the words, including the words
pattern = r'({}).*?'.format('|'.join(words)) * len(words)

print(pattern)

# Find the string between the words
match = re.search(pattern, text, re.DOTALL)
print(match)

if match:
    result = match.group()
    print(result)
else:
    print('No match found')